# Entity Explorer - Account
 <details>
     <summary>&nbsp;<u>Details...</u></summary>

 **Notebook Version:** 1.0<br>
 **Python Version:** Python 3.6 (including Python 3.6 - AzureML)<br>
 **Required Packages**: kqlmagic, msticpy, pandas, numpy, matplotlib, networkx, ipywidgets, ipython, dnspython, ipwhois, folium, maxminddb_geolite2<br>
 **Platforms Supported**:
 - Azure Notebooks Free Compute
 - Azure Notebooks DSVM
 - OS Independent

 **Data Sources Required**:
 - Log Analytics - SecurityAlert, SecurityEvent, HuntingBookmark, Syslog, AAD SigninLogs, AzureActivity, OfficeActivity, ThreatIndicator
 - (Optional) - VirusTotal, AlienVault OTX, IBM XForce, Open Page Rank, (all require accounts and API keys)
 </details>

 Brings together a series of queries and visualizations to help you determine the security state of an Account. The account can be a Windows or Linux account or an Azure Active Directory/Office 365 account.

The Notebook contains sections for reviewing activity for Host accounts (Linux and Windows) and for Azure Active Directory accounts. It also has a general section that looks for related items independent of the account type.

<!DOCTYPE html>
<html>
  <head>
  </head>
  <body>
    <h1>Contents<span class="tocSkip"></span></h1>
    <div class="toc">
      <ul class="toc-item">
        <li><span><a href="#Hunting-Hypothesis">Hunting Hypothesis</a></span></li>
        <li><span><a href="#Notebook-initialization">Notebook initialization</a></span></li>
        <li><span><a href="#Enter-account-name-and-query-time">Enter account
              name and query time</a></span></li>
        <li><span><a href="#Run-the-main-account_summary-function">Run the main
              account_summary function</a></span></li>
        <li><span><a href="#Retrieve-additional-data-for-the-selected-account">Retrieve
              additional data for the selected account</a></span></li>
        <li><span><a href="#Browse-alerts">Browse alerts</a></span></li>
        <li><span><a href="#Browse-other-event-data">Browse other event data</a></span></li>
        <li><span><a href="#Additional-properties-and-methods-of-the-result-object">
              Additional properties and methods of the result object<br>
            </a></span></li>
        <li><span><a href="#Viewing-the-Result-class">Viewing the Result class</a></span></li>
        <li><span><a href="#Use-other-notebooklets-and-pivots-functions-to-drill-down-on-other-entities">
              Use other notebooklets and pivots functions to drill down on other
              entities</a></span></li>
        <li><span><a href="#More-information">More information</a></span></li>
      </ul>
    </div>
    
  </body>
</html>


## Hunting Hypothesis
Our broad initial hunting hypothesis is that a we have received account name entity which is suspected to be compromised and is being used malicious manner in internal networks, we will need to hunt from a range of different positions to validate or disprove this hypothesis.

---
# Notebook initialization
This should complete without errors. If you encounter errors or warnings look at the following notebooks:

- <a href="https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/A%20Getting%20Started%20Guide%20For%20Azure%20Sentinel%20ML%20Notebooks.ipynb">Getting Started Notebook</a>
- [TroubleShootingNotebooks](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/TroubleShootingNotebooks.ipynb)
- [ConfiguringNotebookEnvironment](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)

<details>
    <summary>&nbsp;<u>Details...</u></summary>
The next cell:
- Checks for the correct Python version
- Checks versions and optionally installs required packages
- Imports the required packages into the notebook
- Sets a number of configuration options.

If you are running in the Azure Sentinel Notebooks environment (Azure Notebooks or Azure ML) you can run live versions of these notebooks:
- [Getting Started](./A Getting Started Guide For Azure Sentinel ML Notebooks.ipynb)
- [Run TroubleShootingNotebooks](./TroubleShootingNotebooks.ipynb)
- [Run ConfiguringNotebookEnvironment](./ConfiguringNotebookEnvironment.ipynb)

You may also need to do some additional configuration to successfully use functions such as Threat Intelligence service lookup and Geo IP lookup. 
There are more details about this in the `ConfiguringNotebookEnvironment` notebook and in these documents:
- [msticpy configuration](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)
- [Threat intelligence provider configuration](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html#configuration-file)
</details>

In [1]:
from pathlib import Path
from IPython.display import display, HTML

REQ_PYTHON_VER=(3, 6)
REQ_MSTICPY_VER=(1, 0, 0)

display(HTML("<h3>Starting Notebook setup...</h3>"))
if Path("./utils/nb_check.py").is_file():
    from utils.nb_check import check_versions
    check_versions(REQ_PYTHON_VER, REQ_MSTICPY_VER, extras=["azsentinel"])

# You may need to manually install msticpy with
# !pip install msticpy[azsentinel]

from msticpy.nbtools import nbinit
nbinit.init_notebook(
    namespace=globals(),
    additional_packages=["msticnb>=0.2"],
);


All packages are already installed


### Get Workspace and Authenticate
<details>
    <summary>&nbsp;<u>Details...</u></summary>
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.<br>
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. <br>
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. <br>
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.<br>

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)
```

Note: you may occasionally see a JavaScript warning displayed at the end of the authentication - you can safely ignore this.<br>
On successful authentication you should see a ```popup schema``` button.
To find your Workspace Id go to [Log Analytics](https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.
</details>

In [2]:
# Authentication
# ws_config = WorkspaceConfig()
qry_prov = QueryProvider(data_environment="AzureSentinel")
qry_prov.connect(WorkspaceConfig())

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Authentication and Configuration problems?

If you are having problems, expand the details section below
<br>
<details>
    <summary>Click for details about configuring your authentication parameters</summary>
    
    
The notebook is expecting your Azure Sentinel Tenant ID and Workspace ID to be configured in one of the following places:
- `config.json` in the current folder
- `msticpyconfig.yaml` in the current folder or location specified by `MSTICPYCONFIG` environment variable.
    
For help with setting up your `config.json` file (if this hasn't been done automatically) see the [`ConfiguringNotebookEnvironment`](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb) notebook in the root folder of your Azure-Sentinel-Notebooks project. This shows you how to obtain your Workspace and Subscription IDs from the Azure Sentinel Portal. You can use the SubscriptionID to find your Tenant ID). To view the current `config.json` run the following in a code cell.

```%pfile config.json```

For help with setting up your `msticpyconfig.yaml` see the [Setup](#Setup) section at the end of this notebook and the [ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)
</details>

## Import notebooklets and initialize pivot functions

These are needed for many of the notebook operations

In [3]:
import msticnb as nb

pivot = Pivot(globals())
nb.init(query_provider=qry_prov)

Notebooklets: 9 notebooklets loaded.
Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank


Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank
Notebooklets: Loaded providers: AzureSentinel, geolitelookup, tilookup


# Enter account name and query time window
Type the account name that you want to search for and the time bounds over which you want to search. 

You can specify the account as:

- a simple user name (e.g. `alice`)
- a user principal name (`alice@contoso.com`)
- a qualified windows user name `mydomain\alice`

In the second two cases the domain qualifier will be stripped off before the search. The search is not case sensitive and will match full substrings. E.g. `bob` will match `domain\bob` and `bob@contoso.com` but not `bobg` or `bo`.

In [4]:
account_txt = nbwidgets.GetText(prompt='Enter the Account name to search for:')
md("<hr>")
display(account_txt)
md("<hr>")
pivot.edit_query_time()

Text(value='', description='Enter the Account name to search for:', layout=Layout(width='50%'), style=Descript…

# Run the main *account_summary* function

If more than one matching account name is found, all matches will be shown. 

You can select each one to view more details about the account.

> Note: this is run as a pivot function from the Account entity.
> You can also run it directly as a Notebooklet function by 
> creating and instance of `nb.nblt.azsent.account.AccountSummary()`
> and executing the `run()` method

```python
>>> acct_nb = nb.nblt.azsent.account.AccountSummary()
>>> acct_result = acct_nb.run(pivot.timespan, "account_name")  # subtitute the "account_name" with the name to search
```

In [15]:
Account = entities.Account
acc_result = Account.nblt.account_summary(value=account_txt.value)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Account(Name=-, Host={ 'AdditionalData': {},
  'HostName': 'SOC-FW-RDP',
  'IpAddress': ...)

,TimeGenerated,TenantId,Account,EventID,SourceComputerId,Computer,SubjectUserName,SubjectDomainName,SubjectUserSid,TargetUserName,TargetDomainName,TargetUserSid,TargetLogonId,LogonProcessName,LogonType,LogonTypeName,AuthenticationPackageName,Status,IpAddress,WorkstationName,TimeCreatedUtc,LogonStatus,AccountName,Source
0,2021-06-24 22:22:36.923000+00:00,8ecf8077-cf51-4820-aadd-14040956f35d,\MANAGEDCARE_ADMIN,4625,41502da5-21b7-48ec-81c9-baeea8d7d669,SOC-FW-RDP,-,-,S-1-0-0,MANAGEDCARE_ADMIN,,S-1-0-0,,NtLmSsp,3,3 - Network,NTLM,0xc000006d,193.93.62.10,-,2021-06-24 22:22:36.923000+00:00,failed,MANAGEDCARE_ADMIN,Windows


In [8]:
acc_result.account_selector

<IPython.core.display.Javascript object>

Loading BokehJS ...

<IPython.core.display.Javascript object>

Account(Name=chboeh@microsoft.com, UPNSuffix=microsoft.com, AadTenantId=4b2462a4-bbee-49...)

,TimeGenerated,TenantId,SourceSystem,ResourceId,OperationName,OperationVersion,Category,ResultType,ResultSignature,ResultDescription,DurationMs,CorrelationId,Resource,ResourceGroup,ResourceProvider,Identity,Level,Location,AlternateSignInName,AppDisplayName,AppId,AuthenticationDetails,AuthenticationMethodsUsed,AuthenticationProcessingDetails,AuthenticationRequirement,...,RiskLevelAggregated,RiskLevelDuringSignIn,RiskState,ResourceDisplayName,ResourceIdentity,ServicePrincipalId,ServicePrincipalName,Status,TokenIssuerName,TokenIssuerType,UserAgent,UserDisplayName,UserId,UserPrincipalName,AADTenantId,UserType,FlaggedForReview,IPAddressFromResourceProvider,SignInIdentifier,SignInIdentifierType,ResourceTenantId,HomeTenantId,Type,AccountName,Source
0,2021-06-24 19:56:57.242000+00:00,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,df4a6ae4-7328-4e72-aa4c-e10dc7de2a9e,Microsoft.aadiam,Microsoft.aadiam,,Chris Boehm,4,US,,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,"[\r\n {\r\n ""authenticationStepDateTime"": ""2021-06-24T19:56:57.2425073+00:00"",\r\n ""authe...",,"[\r\n {\r\n ""key"": ""Login Hint Present"",\r\n ""value"": ""True""\r\n },\r\n {\r\n ""key"":...",singleFactorAuthentication,...,none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,,,{'errorCode': 0},,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.447...",Chris Boehm,8c102503-0f98-4b9b-9b4a-ebb56d14c66a,chboeh@microsoft.com,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Guest,None,,,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,72f988bf-86f1-41af-91ab-2d7cd011db47,SigninLogs,chboeh@microsoft.com,AzureActiveDirectory


# Retrieve additional data for the selected account

The result returned from the last cell has a number of properties and methods that you can use
to retrieve and view further information.

The main one for this notebooklet is `get_additional_data`.

In [9]:
acc_result.get_additional_data()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

OperationCount  \
UserPrincipalName    Type       IPAddress      AppResourceProvider                                   UserType                   
chboeh@microsoft.com SigninLogs 208.104.92.208 Amazon Web Services (AWS) -NinjaTenant                Guest                  1   
                                               Azure AD Identity Governance - Entitlement Management Guest                  1   
                                               Azure Portal                                          Guest                 12   
                                               Microsoft 365 Security and Compliance Center          Guest                  1   
                                               Microsoft App Access Panel                            Guest                  1   

                                                                                                                   OperationTypes  \
UserPrincipalName    Type       IPAddress      AppResourceProvider                                   UserType                       
chboeh@microsoft.com SigninLogs 208.104.92.208 Amazon Web Services (AWS) -NinjaTenant                Guest     [Sign-in activity]   
                                               Azure AD Identity Governance - Entitlement Management Guest     [Sign-in activity]   
                                               Azure Portal                                          Guest     [Sign-in activity]   
                                               Microsoft 365 Security and Compliance Center          Guest     [Sign-in activity]   
                                               Microsoft App Access Panel                            Guest     [Sign-in activity]   

                                                                                                               Resources  \
UserPrincipalName    Type       IPAddress      AppResourceProvider                                   UserType              
chboeh@microsoft.com SigninLogs 208.104.92.208 Amazon Web Services (AWS) -NinjaTenant                Guest             1   
                                               Azure AD Identity Governance - Entitlement Management Guest             1   
                                               Azure Portal                                          Guest             1   
                                               Microsoft 365 Security and Compliance Center          Guest             1   
                                               Microsoft App Access Panel                            Guest             1   

                                                                                                                                FirstOperation  \
UserPrincipalName    Type       IPAddress      AppResourceProvider                                   UserType                                    
chboeh@microsoft.com SigninLogs 208.104.92.208 Amazon Web Services (AWS) -NinjaTenant                Guest    2021-06-21 15:06:34.021000+00:00   
                                               Azure AD Identity Governance - Entitlement Management Guest    2021-06-24 14:14:18.669000+00:00   
                                               Azure Portal                                          Guest    2021-06-21 15:03:56.995000+00:00   
                                               Microsoft 365 Security and Compliance Center          Guest    2021-06-24 18:46:52.376000+00:00   
                                               Microsoft App Access Panel                            Guest    2021-06-21 15:06:27.025000+00:00   

                                                                                                                                 LastOperation  
UserPrincipalName    Type       IPAddress      AppResourceProvider                                   UserType                                   
chboeh@microsoft.com SigninLogs 208.104.92.208 Amazon Web Services (AWS) -NinjaTenant        

nan does not appear to be an IPv4 or IPv6 address


,,,,,,Count,FirstOperation,LastOperation
Source,Operation,UserAgent,IpAddress,AsnDescription,CountryCode,,,
AzureActiveDirectory,Sign-in activity,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36 Edg/91.0.864.48",208.104.92.208,"ROCK-HILL-TELEPHONE, US",US,16,2021-06-21 15:03:56.995000+00:00,2021-06-24 19:56:57.242000+00:00


# Browse alerts
If there are any alerts referencing this account name they can be viewed
by calling the `acc_result.browse_alerts()` function.

In [10]:
acc_result.notebooklet.browse_alerts()

,0
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
TimeGenerated,2021-06-22 17:46:22.230000+00:00
AlertDisplayName,Anomalous AAD Account Creation
AlertName,Anomalous AAD Account Creation
Severity,Medium
Description,
ProviderName,ASI Scheduled Alerts
VendorName,Microsoft
VendorOriginalId,9804d24d-bd1d-4814-be58-ff415c63ad72
SystemAlertId,5b67a8d5-fe80-3267-f07e-d39d96aaa901


# Browse other event data

You can use a simple view to make it easier to see the details of individual events
by calling the "view_events" method.

You need to supply the name of the result attribute that you want to view plus
one or more summary columns (as a list of strings).

In [11]:
acc_result.view_events(attrib="azure_activity", summary_cols=["SourceSystem", "Operation", "IPAddress"])

,1
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
SourceSystem,Azure AD
TimeGenerated,2021-06-21 15:03:56.995000+00:00
ResourceId,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam
Operation,Sign-in activity
OperationVersion,1.0
Category,SignInLogs
ResultType,0
ResultSignature,None
DurationMs,0


## Browse events with alternative sorting
You can pass a DatFrame to view events instead of an attribute name.

This means that you can apply sorting or filtering of the data before viewing it.
Here is an example sorting by IPAddress.

In [12]:
acc_result.view_events(
    data=acc_result.azure_activity.sort_values("IPAddress"),
    summary_cols=["SourceSystem", "Operation", "IPAddress"]
)

,20
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
SourceSystem,Azure
TimeGenerated,2021-06-24 19:50:07.300000+00:00
ResourceId,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/Simuland/providers/Microsoft....
Operation,Create/Update Storage Account
Category,Administrative
CorrelationId,b25d12f0-2bdf-4fca-97a2-97eabb279027
Resource,blacksmithsa
ResourceGroup,Simuland
Level,Informational


# Additional properties and methods of the result object

These are static properties - usually DataFrames or visualizations. 
You can access each of these to see or manipulate the retrieved data.

To see help on the available attributes type:
```python
>>> help(acc_result)
```
To see the available methods type:
```python
>>> acc_result.list_methods()
```
> Note, for the AccountSummary notebooklet, the two main data retrieval methods are:<br>
> - run<br>
> - get_additional_data<br>
> There are several other methods that allow you to view individual plots
> or subsets of the data (such as alerts).

To view help on a specific method type:
```python
>>> help(acc_result.method_name())
```

In [ ]:
acc_result.display_alert_timeline()

# Viewing the Result class
You can view all of the data in the results class by "running" it in a cell

> Note: due to the way Jupyter display Javascript objects the plots may
> appear out of order.

In [ ]:
acc_result

# Use other notebooklets and pivots functions to drill down on other entities

You may want to drill down on other entities in the Account data.
You can use methods of the IpAddress or Host entities, for example,
to look at these in more detail.

In [ ]:
IpAddress = entities.IpAddress
ip_result = IpAddress.nblt.ip_address_summary("157.56.162.53")

In [ ]:
ip_result.browse_ti_results()

## Example - getting Whois information for Azure IPAddress requests.

In [ ]:
(
    acc_result                      # the results object
    .azure_activity[["IPAddress"]]  # the property and the column we want
    .drop_duplicates()              # drop duplicates
    .mp_pivot.run(                  # run the pivot function IpAddress 'whois' function
        IpAddress.util.whois, column="IPAddress" 
    )
)

# More information:

## Notebooklets and Pivots
[Notebooklets](https://msticnb.readthedocs.io/en/latest/)

[Pivot functions](https://msticpy.readthedocs.io/en/latest/data_analysis/PivotFunctions.html)

## Notebook/MSTICPy configuration
[MSTICPy Configuration guide](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)

[ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)